<a href="https://colab.research.google.com/github/DanielMerkdo/analitica_marketing/blob/main/c_modelado_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ### Importar librerias
 import numpy as np
 import pandas as pd
 import sqlite3 as sql
 from sklearn.preprocessing import MinMaxScaler
 from ipywidgets import interact
 from sklearn import neighbors
 import joblib

In [ ]:
 ### Conectarse con la base de datos
 conex= sql.connect('data/movies.sqlite')
 cur= conex.cursor()

In [ ]:
 ### Ver las tablas que hay en la base de datos
 cur.execute("SELECT name FROM sqlite_master WHERE type= "table" ")
 print(cur.fetchall())

In [ ]:
 ### Traer tabla de base de datos a Python
 movies= pd.read_sql("""SELECT * FROM movies""", conex)
 ratings= pd.read_sql('SELECT * FROM ratings', conex)

In [ ]:
### Sistemas basados en popular
### Mejores peliculas calificadas que tengan una calificacion diferente de 0.0
pd.read_sql("""select title,
            avg(rating) as avg_rat,
            count(*) as watch_num
            from ratingtitle
            where rating<>0.0
            group by title
            having wacth_num >=15 and watch_num <=100
            order by avg_rat desc
            limit 10
            """, conex)



In [ ]:
### La peliculas mas vistas con promedio
pd.read_sql("""select title,
            avg(iif(rating= 0, Null,rating)) as avg_cal
            count(*) as watch_num
            from ratingtitle
            group by title
            order by watch_num desc
            limit 10
            """, conex)


In [ ]:
### Las mejoras peliculas calificadas segun la fecha en que la vieron
pd.read_sql("""select fecha, title,
            avg(iif(rating= 0.0, Null,rating)) as avg_cal,
            count(iif(rating= 0.0, Null,rating)) as watch_num
            from ratingtitle
            group by title
            having watch_num >=15 and watch_num <=100
            order by avg_cal desc
            limit 10
            """, conex)

In [ ]:
###Sistema de recomendacion basado en contenido de un producto
movies2= pd.read_sql("""select * from movies2""", conex)
movies2= movies2.drop(["level_0", "index"], axis=1)

movies3= pd.read_sql("""select * from movies3""", conex)
movies3= movies3.drop(["index"], axis=1)

In [ ]:
### Imprimir las 10 peliculas mas similares
print(top_similar_movies)

In [ ]:
### funcion de peliculas recomendadas ""
def movierecomendacion(movie= list(movie2["title"])):
    ind_movie= movies2[movies2["title"]== movie].index.values.astype(int)[0]
    similar_movies= movies3.corrwith(movies3.iloc[ind_movie,:], axis=1)
    similar_movies= similar_movies.sort_values(ascending=False)
    top_similar_movies= similar_movies.to_frame(name="correlacion").iloc[0:11]
    top_similar_movies["title"]= movies2["title"]
    return top_similar_movies[["title", "correlacion"]]

In [ ]:
print(interact(movierecomendacion))

In [ ]:
### Sistema de recomendacion basado en contenido KNN un solo producto
model= neighbors.NearestNeighbors(n_neighbors=11, metric="cosine")
model.fit(movies3)
dist, idlist= model.kneighbors(movies3)

distancias= pd.DataFrame(dist)
id_list= pd.DataFrame(idlist)

In [ ]:
def MovieRecomender(movie_name= list(movies2["title"].value_counts().index)):
    movie_list_name= []
    movie_id= movies2[movies2["title"]== movie_name].index
    movie_id= movie_id[0]
    for newid in idlist[movie_id]:
      movie_list_name.append(movies2.loc[newid].title])
    return movie_list_name



In [ ]:
 print(interact(MovieRecomender))